<h1><b>Análise de Compatibilidade de Vagas</b></h1>
<p>Em meio à busca por uma nova oportunidade no mercado de trabalho, é comum encontrar obstáculos que podem tornar o processo frustrante. Desde a concorrência acirrada até os processos seletivos prolongados, diversos fatores podem dificultar essa jornada.</p>
<br>
<p>Com o intuito de simplificar essa busca, desenvolvemos este projeto para agilizar a análise das vagas disponíveis. Aqui, o usuário poderá comparar seu currículo com as oportunidades listadas em uma planilha, identificando aquelas que melhor se alinham ao seu perfil. Dessa forma, é possível direcionar esforços para as vagas mais adequadas, aumentando as chances de sucesso.</p>
<br>
<p>As informações utilizadas neste projeto foram obtidas a partir de fontes públicas, especificamente da seção de Vagas do LinkedIn, sendo empregadas como teste.</p>

<h1><b>Configurações Iniciais</b></h1>

In [1]:
!pip install -U -q google-generativeai

In [30]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [51]:
df = pd.read_csv('lista_de_vagas.csv')
print(df) #Confirmar se os dados foram carregados da planilha

                                                Vaga  \
0                             Engenheiro de Software   
1                Desenvolvedor de Software Embarcado   
2                                 Frontend Developer   
3                      Full Stack Software Developer   
4  Desenvolvedor Front End Pleno (JavaScript/Reac...   
5                                  Desenvolvedor Web   

                                           Descricao  
0  Nessa oportunidade você irá atuar com: Aplicat...  
1  Requisitos: Formação Superior nas áreas de Eng...  
2  Do you want to work with cutting edge technolo...  
3  Sobre a Vaga Estamos buscando um Full Stack So...  
4  Para atuar nessa função, é essencial que você ...  
5  Conhecimento Técnico Experiência comprovada co...  


In [6]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

In [7]:
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE"
}

In [52]:
#Lista de vagas que serão analisadas, transformando o DataFrame em uma lista de dicionários
vagas = df.to_dict(orient='records')

In [49]:
df_vagas = pd.DataFrame(vagas)
df_vagas.columns = ["Vaga", "Descricao"]
df_vagas

,Vaga,Descricao
0,Engenheiro de Software,Nessa oportunidade você irá atuar com: Aplicat...
1,Desenvolvedor de Software Embarcado,Requisitos: Formação Superior nas áreas de Eng...
2,Frontend Developer,Do you want to work with cutting edge technolo...
3,Full Stack Software Developer,Sobre a Vaga Estamos buscando um Full Stack So...
4,Desenvolvedor Front End Pleno (JavaScript/Reac...,"Para atuar nessa função, é essencial que você ..."
5,Desenvolvedor Web,Conhecimento Técnico Experiência comprovada co...


In [15]:
model = "models/embedding-001"

In [53]:
# Função para calcular o percentual de compatibilidade
def calcular_compatibilidade(consulta, vaga, model):
    # Obter o embedding da consulta
    embedding_consulta = genai.embed_content(
        model=model,
        content=consulta,
        task_type="RETRIEVAL_QUERY"
    )["embedding"]

    # Obter o embedding da descrição da vaga
    embedding_vaga = genai.embed_content(
        model=model,
        content=vaga["Descricao"],
        title=vaga["Vaga"],
        task_type="RETRIEVAL_DOCUMENT"
    )["embedding"]

    # Calcular a similaridade de cosseno entre os embeddings
    similaridade_cosseno = np.dot(embedding_consulta, embedding_vaga) / (np.linalg.norm(embedding_consulta) * np.linalg.norm(embedding_vaga))

    # Converter a similaridade de cosseno em percentual
    percentual_compatibilidade = (similaridade_cosseno + 1) * 50

    return percentual_compatibilidade

<h1><b>Análise dos Dados</b></h1>

In [50]:
consulta = input("Por favor, insira seu currículo: ")

# Calcular o percentual de compatibilidade para cada vaga
df_vagas["Percentual de Compatibilidade"] = df_vagas.apply(lambda row: calcular_compatibilidade(consulta, row, model), axis=1)

# Exibição da nova tabela com o nome da vaga, descrição e percentual de compatibilidade
df_vagas

Por favor, insira seu currículo: Experiência Logo da empresa Confidencial Desenvolvedor Front End Jr Desenvolvedor Front End Jr Confidencial · Tempo integral Confidencial · Tempo integral ago de 2023 - o momento · 10 meses De ago de 2023 até o momento · 10 meses Atibaia, São Paulo, Brasil · Remota Atibaia, São Paulo, Brasil · Remota Desenvolvimento de integrações e programação de aplicações web e websites. Levantamento de novas implementações, avaliando o impacto nas funcionalidades existentes. Desenvolvimento de integrações e programação de aplicações web e websites. Levantamento de novas implementações, avaliando o impacto nas funcionalidades existentes. WordPress, Scrum e mais 10 competências Logo da empresa Confidencial Desenvolvedor Front End Freelancer Desenvolvedor Front End Freelancer Confidencial · Freelance Confidencial · Freelance nov de 2022 - o momento · 1 ano 7 meses De nov de 2022 até o momento · 1 ano 7 meses Remota Remota Desenvolvimento de aplicações e páginas web, co

,Vaga,Descricao,Percentual de Compatibilidade
0,Engenheiro de Software,Nessa oportunidade você irá atuar com: Aplicat...,83.857909
1,Desenvolvedor de Software Embarcado,Requisitos: Formação Superior nas áreas de Eng...,83.228585
2,Frontend Developer,Do you want to work with cutting edge technolo...,80.630244
3,Full Stack Software Developer,Sobre a Vaga Estamos buscando um Full Stack So...,82.611128
4,Desenvolvedor Front End Pleno (JavaScript/Reac...,"Para atuar nessa função, é essencial que você ...",85.819456
5,Desenvolvedor Web,Conhecimento Técnico Experiência comprovada co...,86.087586
